In [333]:
import os
import pprint
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

SCOPE = ['https://spreadsheets.google.com/feeds']
OAUTH = 'gsbook/gsbook-oauth.json'

import gsbook.worker as worker

In [424]:
event_df = pd.read_csv('tests/test_data', index_col=0)
event_df

,Date,Name,Standard,Version,Score,Remark
0,11/21/2016,Alex,A,1,0,sick
1,11/21/2016,Ada,A,1,1,NaN
2,11/21/2016,John,A,1,1,NaN
3,11/21/2016,George,A,1,1,NaN
4,11/21/2016,Sam,A,1,2,NaN
5,11/21/2016,Paul,A,1,2,NaN
6,11/21/2016,Kay,A,1,2,NaN
7,11/21/2016,Zack,A,1,3,NaN
8,11/21/2016,Mary,A,1,3,NaN
9,11/22/2016,Alex,A,2,0,NaN


In [425]:
summary_df = event_df.groupby(['Name','Standard']).max()['Score'].unstack()
summary_df

Standard,A,B
Name,,
Ada,4,1
Alex,4,0
George,1,1
John,1,1
Kay,2,2
Mary,3,4
Paul,2,2
Sam,2,2
Zack,3,3


In [426]:
standard_df = event_df.groupby(['Standard', 'Version']).count()['Date'].unstack().fillna(0)
standard_df

Version,1,2
Standard,,
A,11.0,9.0
B,0.0,10.0


In [442]:
person_df = event_df.groupby(['Name','Standard','Version','Date']).agg({'Score':['max','count'], 'Remark':['sum']})[['Score', 'Remark']]
person_df

Score            Remark
                                     max count         sum
Name   Standard Version Date                              
Ada    A        1       11/21/2016     1     1         NaN
                        11/23/2016     4     1         NaN
                2       11/22/2016     1     1         NaN
       B        2       11/25/2016     1     1         NaN
Alex   A        1       11/21/2016     0     1        sick
                        11/23/2016     4     1      retake
                2       11/22/2016     0     1         NaN
       B        2       11/24/2016     0     1         NaN
George A        1       11/21/2016     1     1         NaN
                2       11/22/2016     1     1         NaN
       B        2       11/27/2016     1     1         NaN
John   A        1       11/21/2016     1     1         NaN
                2       11/22/2016     1     1         NaN
       B        2       11/26/2016     1     1         NaN
Kay    A        1       11/21/2016     2     1         NaN
                2       11/22/2016     2     1         NaN
       B        2       11/30/2016     2     1         NaN
Mary   A        1       11/21/2016     3     1         NaN
                2       11/22/2016     3     1         NaN
       B        2       12/2/2016      4     2  sickretake
Paul   A        1       11/21/2016     2     1         NaN
                2       11/22/2016     2     1         NaN
       B        2       11/29/2016     2     1         NaN
Sam    A        1       11/21/2016     2     1         NaN
                2       11/22/2016     2     1         NaN
       B        2       11/28/2016     2     1         NaN
Zack   A        1       11/21/2016     3     1         NaN
                2       11/22/2016     3     1         NaN
       B        2       12/1/2016      3     1         NaN

In [431]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(OAUTH, SCOPE)
gc = gspread.authorize(credentials)
sheet = gc.open('Math000_SBG')

In [433]:
sheet.title

'Math000_SBG'

In [454]:
event_df.groupby(['Name','Standard','Version']).max().unstack(level=0)['Score'].transpose()

Standard  A     B
Version   1  2  2
Name             
Ada       4  1  1
Alex      4  0  0
George    1  1  1
John      1  1  1
Kay       2  2  2
Mary      3  3  4
Paul      2  2  2
Sam       2  2  2
Zack      3  3  3